In [62]:
# タイムスタンプ + MACアドレスのデータを
# MACアドレス, in時間, out時間に変換してCSVで出力する

import pandas as pd

# csvファイルの読み込み
directory = 'csv/Chofu/'
filename = 'seiyu'
df = pd.read_csv(directory + filename + '.csv')

# 無効なデータが入っている1行目を削除
df = df.drop(0, axis=0)

# 日付をintに変換
df_int_cast = df.astype({'year': int, 'month': int, 'day': int, 'hour': int, 'minute': int, 'second': int})

# コピー
df_new = df_int_cast

# df_int_cast

# 時刻を秒に変換
days = 0 #df_int_cast['year'] * 365 + df_int_cast['month'] * 12 + df_int_cast['day']
hours = days * 24 + df_int_cast['hour']
minutes = hours * 60 + df_int_cast['minute']
df_new['elapsed_time'] = minutes * 60 + df_int_cast['second'] 

# 時刻の列を削除
df_address_time = df_new.drop(['year', 'month', 'day', 'hour', 'minute', 'second'], axis=1)

# Bleデータを管理するクラス
# addressが同じで前回計測されてからpersistence以内であればout_timeを更新する
# そうでなければFalseを返し、外部で新規BLEデバイスとしてリストに追加する
class BleLog:
    def __init__(self, address, time, persistence):
        self.address = address
        self.in_time = time 
        self.out_time = time
        self.persistence = persistence
        
    def update(self, address, time):
        
        # check address
        if self.address != address:
            return False
        
        # check elapsed time
        if time - self.out_time > self.persistence:
            return False
        
        # update out time
        self.out_time = time
        
        return True
    
    def print_data(self):
        start_h = int(self.in_time / 3600)
        start_m = int((self.in_time - start_h * 3600) / 60)
        start_s = int(self.in_time - start_h * 3600 - start_m * 60)
        end_h = int(self.out_time / 3600)
        end_m = int((self.out_time - end_h * 3600) / 60)
        end_s = int(self.out_time - end_h * 3600 - end_m * 60)
#         print(self.address + " " + str(self.in_time) + " " + str(self.out_time) + " " + str(self.out_time - self.in_time))
        diff = self.out_time - self.in_time
        print('address: %s | start %02s:%02d:%02d -> end %02s:%02d:%02d | time: %d' %(self.address, start_h, start_m, start_s, end_h, end_m, end_s, diff))
        

# 現在のリストと比較する
def update_ble_data(address, time, ble_list):
    for ble in ble_list:
        if ble.update(address, time):
            return True
    
    return False

# bleデータを表示する
def print_ble_data(data):
    for ble in data:
        ble.print_data()

# リストを初期化
ble_list_a = []

# csvからデータを読み込みbleリストと比較していく
# リストになければ新規にbleデバイスを追加
for index, row in df_address_time.iterrows():
    addr = row['address']
    time = row['elapsed_time']
    
    if not update_ble_data(addr, time, ble_list_a):
        new_ble = BleLog(addr, time, 120)
        ble_list_a.append(new_ble)

# 出力用のDataFrameに書き出す
addresses = []
in_times = []
out_times = []
in_times_str = []
out_times_str = []

for data in ble_list_a:
    addresses.append(data.address)
    in_times.append(data.in_time)
    out_times.append(data.out_time)
    
    start_h = int(data.in_time / 3600)
    start_m = int((data.in_time - start_h * 3600) / 60)
    start_s = int(data.in_time - start_h * 3600 - start_m * 60)
    end_h = int(data.out_time / 3600)
    end_m = int((data.out_time - end_h * 3600) / 60)
    end_s = int(data.out_time - end_h * 3600 - end_m * 60)
    
    start_str = '{0:02d}'.format(start_h) + ':' + '{0:02d}'.format(start_m) + ':' + '{0:02d}'.format(start_s)
    end_str = '{0:02d}'.format(end_h) + ':' + '{0:02d}'.format(end_m) + ':' + '{0:02d}'.format(end_s)
    
    in_times_str.append(start_str)
    out_times_str.append(end_str)
    
out_df = pd.DataFrame({ 'address': addresses,
                        'in': in_times,
                        'out': out_times,
                        'in_time': in_times_str,
                        'out_time': out_times_str})

# out_df

out_df.to_csv(directory + 'result/r_' + filename + '.csv')

In [54]:
# 結果を表示
print_ble_data(ble_list_a)

address: 48:98:4E:98:50:95 | start 17:55:00 -> end 17:56:03 | time: 63
address: 5C:C1:AC:FE:C8:05 | start 17:55:00 -> end 18:14:41 | time: 1181
address: 42:BF:4F:7C:4E:C8 | start 17:55:00 -> end 17:55:38 | time: 38
address: 66:E0:62:EF:F4:F7 | start 17:55:00 -> end 18:14:56 | time: 1196
address: 45:51:6B:60:41:A4 | start 17:55:00 -> end 17:55:33 | time: 33
address: 63:04:03:68:5B:AA | start 17:55:00 -> end 17:56:04 | time: 64
address: 4B:F5:89:62:8A:E1 | start 17:55:00 -> end 17:56:40 | time: 100
address: 59:63:C1:B7:6B:B2 | start 17:55:00 -> end 18:00:08 | time: 308
address: 7E:CA:B0:0C:69:2E | start 17:55:00 -> end 17:55:36 | time: 36
address: 77:81:CD:CC:EE:41 | start 17:55:00 -> end 17:55:35 | time: 35
address: 7A:28:F3:6D:AE:5A | start 17:55:00 -> end 18:02:11 | time: 431
address: 76:26:63:44:98:B8 | start 17:55:00 -> end 18:08:43 | time: 823
address: 78:B0:0B:F4:9A:EC | start 17:55:00 -> end 17:58:16 | time: 196
address: 6B:13:A6:61:E9:5B | start 17:55:00 -> end 17:55:20 | time: 

In [12]:
len(ble_list_a)

829

In [79]:
# もう一つのデータを読み込む
df_b = pd.read_csv('csv/20190227_kanda_b.csv')

# 無効なデータが入っている1行目を削除
df_b = df_b.drop(0, axis=0)

# 日付をintに変換
df_int_cast_b = df_b.astype({'year': int, 'month': int, 'day': int, 'hour': int, 'minute': int, 'second': int})

# コピー
df_new_b = df_int_cast_b

In [80]:
# 時刻を秒に変換
days = 0 #df_int_cast['year'] * 365 + df_int_cast['month'] * 12 + df_int_cast['day']
hours = days * 24 + df_int_cast_b['hour']
minutes = hours * 60 + df_int_cast_b['minute']
df_new_b['elapsed_time'] = minutes * 60 + df_int_cast_b['second'] 

# 時刻の列を削除
df_address_time_b = df_new_b.drop(['year', 'month', 'day', 'hour', 'minute', 'second'], axis=1)

In [81]:
# リストを初期化
ble_list_b = []

# csvからデータを読み込みbleリストと比較していく
# リストになければ新規にbleデバイスを追加
for index, row in df_address_time_b.iterrows():
    addr = row['address']
    time = row['elapsed_time']
    
    if not update_ble_data(addr, time, ble_list_b):
        new_ble = BleLog(addr, time, 120)
        ble_list_b.append(new_ble)

In [82]:
len(ble_list_b)

743

In [83]:
count_A = 0
count_B = 0
count_AB = 0
count_BA = 0

# list a と list bを比較する
for a in ble_list_a:
    addr_a = a.address
    in_a = a.in_time
    only_a = True
    
    for b in ble_list_b:
        addr_b = b.address
        in_b = b.in_time
        
        if addr_a == addr_b:
            only_a = False
            if in_a < in_b:
                count_AB += 1
            else:
                count_BA += 1
            continue
        
    if only_a:
        count_A += 1

count_B = len(ble_list_b) - count_AB - count_BA


In [84]:
count_A

282

In [85]:
count_B

263

In [86]:
count_AB

262

In [87]:
count_BA

218

In [88]:
len(ble_list_b)

743

In [89]:
len(ble_list_a)

725